In [23]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re

page = 1
BASE_URL = 'https://www.morungexpress.com/'
country = 'bangladesh'
initial_url = f'{BASE_URL}search?keyword={country}&page={page}'

all_news_links = []

#COLLECTING ALL LINKS

#USE WHILE LOOP TO FIND ALL PAGES. 
#AT THE TIME OF SCRAPING, THERE ARE 255 PAGES REGARDING BANGLADESH.

#FIRST 3 PAGES TAKEN FOR SCRAPING.

#while True
for page in range(1,4):

    initial_url = f'{BASE_URL}search?keyword={country}&page={page}'

    response = requests.get(initial_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    all_news_lists = soup.find_all('li', class_ = 'view-list-item m-md-3 p-3')
    
    if len(all_news_lists) > 0:
    
        for each_news in all_news_lists:

            all_news_links.append(each_news.find('div', class_ = 'post-title').find('a').get('href'))
    
    else:
        
        break
        
    #page+=1



#SCRAPING NEWS DATA
data_list = []
counter = 0

#SCRAPING PROCESS VERY LENGTHY FOR EACH LINK. SO, FIRST 16 LINKS TAKEN FOR THIS SCRIPT. 

for url in all_news_links[:16]:


        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        #TITLE

        title_tag = soup.find('h1', class_ = 'text-dark title-green-line-left')
        title = title_tag.text.strip() if title_tag else 'Title not found'
        

        #DATE

        #NO INFORMATION ABOUT TIME GIVEN IN THIS WEBSITE

        date_info = soup.find('span', class_ = 'post-created')

        if date_info:

            date_data = date_info.find('span').text.strip()
    
    
    
            try:
                source_localtime = datetime.strptime(date_data.strip(), '%dth %B %Y')

            except ValueError:

                try:
                    source_localtime = datetime.strptime(date_data.strip(), '%dst %B %Y')

                except ValueError:
            

                    try:
                        source_localtime = datetime.strptime(date_data.strip(), '%dnd %B %Y')

                    except ValueError:

                        try:
                            source_localtime = datetime.strptime(date_data.strip(), '%drd %B %Y')
                    
                        except:
                            source_localtime = 'None'
                            date_data = 'Date data not found'
                            
            

        #CONTENT

        content = []

        main_section = soup.find('section', {'id':'specialities'})

        if main_section:

            main_div = soup.find_all('div', class_ = 'post_content')

            if main_div:

                all_paras = main_div[1].find_all('p')

                for each_para in all_paras:

                    content.append(each_para.text)

                full_content = ''.join(content)



                full_content = re.sub('\n|\r', '', full_content)
                full_content = full_content.replace('\xa0','')

                #AUTHOR
                author_tag = all_paras[0].find('strong').text
                
                
                author_split = author_tag.split('(')

                if len(author_split) > 1:

                    author = author_split[1]
                    author_with_brackets = '('+author
                    author = re.sub(r'[^\w\s]', '', author)

                    author = author.strip()
                    
                    if len(author) > 20:
                        
                        author_strong_elements = all_paras[0].find_all('strong')          
                            
                        author = author_strong_elements[1].text
                    

                    temp_content = full_content.split(author_with_brackets.strip())

                    if len(temp_content) > 1:
                        full_content = temp_content[1]
                    else:
                        full_content = temp_content[0]


                else:
                    author = author_split[0] 

                    author = re.sub(r'[^\w\s]', '', author)

                    author = author.strip()
                    
                    if len(author) > 20:
                        
                        author_strong_elements = all_paras[0].find_all('strong')
                        
                        author = author_strong_elements[1].text
                    

                    temp_content = full_content.split(author)

                    if len(temp_content) > 1:
                        full_content = temp_content[1]
                    else:
                        full_content = temp_content[0]



                full_content = full_content.strip()

            else:
                full_content = 'Content not found'
                author = 'Author not found'



        else:
            full_content = 'Content not found'
            author = 'Author not found'
            
        
        #NO CONTENT SUMMARIES IN THIS WEBSITE
        
        content_summary = 'None'
        summary_translation = 'None'
        title_translation = 'None'
        content_translation = 'None'
#         source_localtime = 'None'
        bangladesh_localtime = source_localtime

        data_dict = {
        "url": url,
        "title": title,
        "content": full_content,
        "content_summary": content_summary,
        "title_translation":title_translation,
        "content_translation":content_translation,
        "summary translation":summary_translation,
        "author": author,
        "country": country,
        'source_localtime': source_localtime,
        'bangladesh_localtime': bangladesh_localtime

        }

        counter+=1


        if (full_content != "Content not Found" and title != 'Title not found'):

                if data_dict not in data_list:
                    # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')
            

df = pd.DataFrame(data_list)
df.head()

csv_filename = f"{country}_The_Morung_Express.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["date"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="bangladesh_localtime", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)

Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added
Link 11 added
Link 12 added
Link 13 added
Link 14 added
Link 15 added
Link 16 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://www.morungexpress.com/from-asia-to-afr...,"From Asia to Africa, 'Vishwaguru' India plays ...",In yet another proof of its emergence as an em...,None,None,None,None,IANS,bangladesh,2024-03-02,2024-03-02
1,https://www.morungexpress.com/nagaland-coal-re...,Nagaland coal revenue plummets: Rs 6 crore sho...,Kohima | March 1The revenue generated from coa...,None,None,None,None,Morung Express News,bangladesh,2024-03-01,2024-03-01
2,https://www.morungexpress.com/nitin-gadkari-sa...,"Nitin Gadkari sanctions Rs 3,371cr for 4-lanin...",Union Minister for Road Transport and Highways...,None,None,None,None,IANS,bangladesh,2024-03-01,2024-03-01
3,https://www.morungexpress.com/central-contract...,Central Contracts: Hardik Pandya’s case is dif...,Former Indian cricketer-turned-commentator Aak...,None,None,None,None,IANS,bangladesh,2024-03-01,2024-03-01
4,https://www.morungexpress.com/death-toll-in-ba...,Death toll in Bangladesh building fire rises t...,The death toll in the fire at a multi-storey b...,None,None,None,None,IANS,bangladesh,2024-03-01,2024-03-01
